In [3]:
import datetime

#On crée une fonction pour récupérer les bases de données grâce à l'API de la sncf
import requests
import pandas as pd

def recup_base(ident) :
    #req = requests.get('https://ressources.data.sncf.com/api/v2/catalog/datasets/enquetes-gares-connexions-repartition-homme-femme/exports/json?where=date_format%28annee%2C%27YYYY%27%29%20%3D%20%272017%27&limit=-1&offset=0&timezone=UTC')
    req = requests.get('https://ressources.data.sncf.com/api/v2/catalog/datasets/'+ident+'/exports/json?limit=-1&offset=0&timezone=UTC')
    wb = req.json()
    wb = pd.json_normalize(wb)
    return(wb)

In [4]:
#On importe les bases de données nécessaires à notre étude
mode_acces=recup_base('repartition-de-lutilisation-des-moyens-dacces-ou-de-diffusion-a-la-gare-enquetes')
motif_dep=recup_base('repartition-des-motifs-de-deplacements-des-voyageurs-et-non-voyageurs-enquetes-e')
distance_parcourue=recup_base('distance-parcourue')
freq_gare=recup_base('frequence-gare')
intermodalite = recup_base('intermodalite')
wifi=recup_base('gares-equipees-du-wifi')
travaux=recup_base('travaux-dans-les-grandes-gares') #secondaire
temps_prec=recup_base('temps-de-precaution-en-gare')
type_clients=recup_base('types-de-clients-dans-les-gares') #secondaire
csp_voyageurs=recup_base('enquetes-gares-connexions-repartition-par-repartition-par-categories-socio-profe')
age_voyageurs=recup_base('enquetes-gares-connexions-repartition-repartition-par-classe-dage')
genre_voyageurs=recup_base('enquetes-gares-connexions-repartition-homme-femme')
referentiel_gare = recup_base('referentiel-gares-voyageurs')
frequentation_gares=recup_base('frequentation-gares')

In [7]:
#Pour chaque base, on vérifie que chaque gare n'est enquêté qu'une seule fois (verif permet de contrôler si chaque gare doit apparaître plusieurs fois dans la base de données, comme pour une enquête sur les CSP des voyageurs)
def check_base(base,gare,verif) :
    a=base[[gare,verif]].drop_duplicates().size
    b=base[[gare,verif]].size
    return(a==b)

#Renvoie un booléen indiquant que chaque doublet gare/catégorie n'est présent au plus qu'une fois. S'il n'y a pas de catégorie, verif devient gare

def list_gare(base,gare) :
    return(base[gare].drop_duplicates())


#On crée une fonction qui supprime les enquêtes qui ne sont pas l'enquête la plus récente dans la base
def nettoyage_multi_enquete(base,gare,index,annee) :
    duplicate=base[base[[gare,index]].duplicated(keep=False)]
    duplicate
    liste_gare=duplicate[gare].unique()
    liste_gare
    copie=base.copy()
    for g in liste_gare :
        occ=copie[copie[gare]==g]
        an=pd.to_numeric(occ[annee]).max()
        copie.drop( copie[ (copie[gare] == g) & (pd.to_numeric(copie[annee]) < an) ].index, inplace=True)
    return(copie)    
    
    
check_base(wifi,"uic","service_wifi")
wifi
frequentation_gares
#check_base(type_clients,"gare_enquetee","type_d_usager")
#type_clients[["gare_enquetee","type_d_usager"]].drop_duplicates().size
#type_clients[["gare_enquetee","type_d_usager"]]

,nom_gare,code_uic_complet,code_postal,segmentation_drg_2021,total_voyageurs_2020,total_voyageurs_non_voyageurs_2020,total_voyageurs_2019,total_voyageurs_non_voyageurs_2019,total_voyageurs_2018,total_voyageurs_non_voyageurs_2018,totalvoyageurs2017,total_voyageurs_non_voyageurs_2017,total_voyageurs_2016,total_voyageurs_non_voyageurs_2016,total_voyageurs_2015,total_voyageurs_non_voyageurs_2015
0,Abancourt,87313759,60220,C,32396,32396,42685,42685,40228,40228,43760,43760,41096,41096,39720,39720
1,Achères Grand Cormier,87386052,78100,B,120871,120871,221003,221003,228032,228032,235698,235698,238687,238687,241046,241046
2,Acheux - Franleu,87316745,80560,C,0,0,2,2,235,235,104,104,308,308,898,898
3,Agen,87586008,47000,A,860964,1076205,1211323,1514154,1109199,1386499,1194455,1493068,1141620,1427026,1183150,1478938
4,Alaï,87722744,69340,C,32702,32702,37840,37840,37031,37031,35720,35720,27939,27939,21972,21972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,Vorey,87734723,43800,C,29492,29492,45135,45135,29189,29189,24157,24157,35397,35397,34581,34581
2963,Vulaines-sur-Seine - Samoreau,87682443,77870,C,28791,28791,52885,52885,54449,54449,55824,55824,56647,56647,57163,57163
2964,Weyersheim,87213678,67720,B,130434,163043,164563,205703,155497,194371,155940,194925,157555,196944,160298,200372
2965,Yffiniac,87473116,22120,C,19066,19066,28784,28784,27911,27911,33222,33222,39188,39188,36880,36880


In [5]:
#Reste à traiter : wifi, proprete,
def dummy_var(base,index,var) :
    if check_base(base,index,var) :
        return(pd.get_dummies(base[[index,var,'pourcentage']].set_index(index), var).copy().reset_index())
    else :
        return ("error")

def dummy_bin(base,index,var) :
    if check_base(base,index,var) :
        return(pd.get_dummies(base[[index,var]].set_index(index), var).copy())
    else :
        return ("error") 
#def nettoyage_rec(base,index,var) :
 #   doublons=base[[index,var]].duplicated()
 #   for i in range(length(doublons))
        
    
mode_acces_dm=dummy_var(mode_acces,'uic','gpe_intermodalites_usagers')
motif_dep_dm=dummy_var(motif_dep,'uic','motif_du_deplacement')
distance_parcourue_dm=dummy_var(distance_parcourue,'uic','classe_distance_gare_gare')
freq_gare_dm=dummy_var(freq_gare,"uic","frequence")
intermodalite_dm=dummy_var(intermodalite,"uic","gpe_intermodalites_usagers")
temps_prec_dm=dummy_var(temps_prec,"uic","temps_de_precaution")
type_clients_dm=dummy_var(type_clients,"uic","type_d_usager")

#Gestion des deux bases ayant des gares avec plusieurs enquêtes
csp_voyageurs_dm=dummy_var(nettoyage_multi_enquete(csp_voyageurs,"uic","csp",'annee'),"uic","csp")
age_voyageurs_dm=dummy_var(nettoyage_multi_enquete(age_voyageurs,"uic","classe_d_age","annee"),"uic","classe_d_age")


genre_voyageurs_dm=dummy_bin(genre_voyageurs,"uic","sexe")
wifi_dm=dummy_bin(wifi,"uic","service_wifi")

#On modifie le format de date dans proprete pour nettoyer les anciennes enquêtes
#def change_date(date) : 
#    return(datetime.datetime.strptime(date,'%Y-%m'))
#datetime.datetime.strptime('2017-03','%Y-%m')<datetime.datetime.strptime('2017-02','%Y-%m')

#proprete["mois"]=proprete["mois"].apply(change_date)

In [10]:
mode_acces.to_excel(r'Projet-Python-SNCF\mode_acces.xlsx')
motif_dep.to_excel(r'Projet-Python-SNCF\motif_dep.xlsx')
distance_parcourue.to_excel(r'Projet-Python-SNCF\distance_parcourue.xlsx')
freq_gare.to_excel(r'Projet-Python-SNCF\freq_gare.xlsx')
intermodalite.to_excel(r'Projet-Python-SNCF\intermodalite.xlsx')
wifi.to_excel(r'Projet-Python-SNCF\wifi.xlsx')
travaux.to_excel(r'Projet-Python-SNCF\travaux.xlsx')
temps_prec.to_excel(r'Projet-Python-SNCF\temps_prec.xlsx')
type_clients.to_excel(r'Projet-Python-SNCF\type_clients.xlsx')
csp_voyageurs.to_excel(r'Projet-Python-SNCF\csp_voyageurs.xlsx')
age_voyageurs.to_excel(r'Projet-Python-SNCF\age_voyageurs.xlsx')
genre_voyageurs.to_excel(r'Projet-Python-SNCF\genre_voyageurs.xlsx')
referentiel_gare.to_excel(r'Projet-Python-SNCF\referentiel_gare.xlsx')
frequentation_gares.to_excel(r'frequentation_gares.xlsx')

,uic,gare_enquetee,classe_distance_gare_gare,pourcentage,annee
0,142109.0,Lorraine TGV,Moins de 25 km,1.0,2017
1,118000.0,Troyes,250 à 300 km,2.0,2015
2,192039.0,Metz,100 à 150 km,5.0,2016
3,171009.0,Reims,250 à 300 km,2.0,2016
4,611004.0,Toulouse Matabiau,100 à 150 km,13.0,2014
...,...,...,...,...,...
492,763029.0,Valence TGV Rhône-Alpes Sud,300 à 400 km,3.0,2015
493,773002.0,Montpellier Saint-Roch,150 à 200 km,7.0,2015
494,747006.0,Grenoble,250 à 300 km,0.0,2015
495,747006.0,Grenoble,100 à 150 km,2.0,2015
